# 0) Prepare base dataframes
    - It cotains import tools, fix seed, load datasets, feature selection for only using interesting variables, etc.. 

    - eICU는 MIMIC과 달리 `offset`이 분 단위로 기록이 되어 있어 시간 관련 변수를 처리할 때 초 단위로 굳이 바꾸지 않고 분 단위를 그대로 활용

    - eICU의 lab 데이터는 자체에 `patientunitstayid` = `stay_id`가 존재하여 따로 stay_id를 할당해주는 작업 불필요

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import random 
import os 
import gc
import math
import json
import optuna
import re
import warnings

from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\korea\miniconda3\envs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def fix_seed(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 

my_seed = 42
fix_seed(my_seed)

In [3]:
with open('./utils/concept-dict.json', 'r') as f:
    concept_dict = json.load(f)

static_vars = ["age", "sex", "height", "weight"]

dynamic_vars = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir",
                  "bnd", "bun", "ca", "cai", "ck", "ckmb", "cl", "crea", "crp", 
                  "dbp", "fgn", "fio2", "glu", "hgb", "hr", "inr_pt", "k", "lact",
                  "lymph", "map", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", 
                  "o2sat", "pco2", "ph", "phos", "plt", "po2", "ptt", "resp", "sbp", 
                  "temp", "tnt", "urine", "wbc"]

# concept_dict는 다음과 같은 구조로 되어있음 

"""

변수 이름 > unit, min, max, category

-- source 정보 
ids : 해당 변수 고유 식별 번호
table : 어떤 테이블에 존재하는지?
sub_var : 식별 번호가 table의 어떤 column에서 조회해야 하는지?
call_back : 어떠한 전처리를 고려했는지? (e.g.) 단위 변환등 어떤 전처리를 수행해야 하는지 서술해놓음
"""
concept_dict[dynamic_vars[16]]['sources']['eicu']

[{'ids': 'CRP',
  'table': 'lab',
  'sub_var': 'labname',
  'callback': "convert_unit(binary_op(`*`, 10), 'mg/L', 'mg/dl')"}]

In [4]:
# 해당 데이터셋마다 필요한 값들을 excel 형식으로 변환해보자.

def select_vars(db_name, var_list, base_dict):
    """
    min, max => 이상치 처리 시 사용
    cateogries => 해당 변수 category 분류 시 사용
    source_table => 어느 테이블에서 가져와야 하는지
    source_columns => 해당 테이블 어느 컬럼에서 가져와야 하는지
    source_itemid => 해당 테이블의 컬럼에서 어떤 id로 존재하는지
    """
    var_normal_min = []
    var_normal_max = []
    var_unit = []
    source_category = []
    source_table = []
    source_column = []
    source_itemid = []
    source_callback = []
    source_regex = []
    source_vars = [] # to map other lists
    for var in tqdm(var_list):

        for i in range(len(base_dict[var]['sources'][db_name])): # 여러 테이블에 분산되어 있는 경우 길이가 2가 넘을 수 있기 때문

            source_table.append(base_dict[var]['sources'][db_name][i]['table'])

            try:
                source_column.append(base_dict[var]['sources'][db_name][i]['sub_var'])
            except:
                source_column.append(base_dict[var]['sources'][db_name][i]['val_var'])

            try:
                source_itemid.append(base_dict[var]['sources'][db_name][i]['ids'])
            except:
                source_itemid.append(None)

            try:
                source_callback.append(base_dict[var]['sources'][db_name][i]['callback'])
            except:
                source_callback.append(None)
            
            try:
                source_regex.append(base_dict[var]['sources'][db_name][i]['regex'])
            except:
                source_regex.append(None)

            source_category.append(base_dict[var]['category'])

            try:
                var_unit.append(base_dict[var]['unit'])
            except:
                var_unit.append(None)

            try:
                var_normal_min.append(base_dict[var]['min'])
            except:
                var_normal_min.append(None)
            
            try:
                var_normal_max.append(base_dict[var]['max'])
            except:
                var_normal_max.append(None)


            source_vars.append(var)
        
    return var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex

var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex  = select_vars('eicu', static_vars + dynamic_vars, concept_dict)

MAPPING_DF = pd.DataFrame({
    'var_name' : source_vars,
    'normal_min' : var_normal_min,
    'normal_max' : var_normal_max,
    'category' : source_category,
    'table' : source_table,
    'column' : source_column,
    'itemid' : source_itemid,
    'unit' : var_unit,
    'method' : source_callback,
    'regex' : source_regex
})

MAPPING_DF

100%|██████████| 52/52 [00:00<?, ?it/s]


,var_name,normal_min,normal_max,category,table,column,itemid,unit,method,regex
0,age,0.0,100.0,demographics,patient,age,None,years,transform_fun(eicu_age),None
1,sex,NaN,NaN,demographics,patient,gender,None,None,None,None
2,height,10.0,230.0,demographics,patient,admissionheight,None,cm,None,None
3,weight,1.0,500.0,demographics,patient,admissionweight,None,kg,None,None
4,alb,0.0,6.0,chemistry,lab,labname,albumin,g/dL,None,None
5,alp,0.0,NaN,chemistry,lab,labname,alkaline phos.,"[IU/L, U/l]",None,None
6,alt,0.0,NaN,chemistry,lab,labname,ALT (SGPT),"[IU/L, U/l]",None,None
7,ast,0.0,NaN,chemistry,lab,labname,AST (SGOT),"[IU/L, U/l]",None,None
8,be,-25.0,25.0,blood gas,lab,labname,Base Excess,"[mEq/L, mmol/l]",None,None
9,bicar,5.0,50.0,chemistry,lab,labname,bicarbonate,"[mEq/L, mmol/l]",None,None


In [7]:
# Original Data Load..
ROOT_DIR = '/Users/korea/datasets/physionet.org/files/eicu-crd/2.0'
ROOT_DIR = Path(ROOT_DIR)

# Make Output Folder
if not os.path.exists(ROOT_DIR/'preprocessed_yaib'):
	os.makedirs(ROOT_DIR/'preprocessed_yaib')

## ICU
# admissiondrug_origin = pd.read_csv(ROOT_DIR/'admissionDrug.csv.gz',compression = 'gzip')
# admissiondx_origin = pd.read_csv(ROOT_DIR/'admissionDx.csv.gz',compression = 'gzip')
# allergy_origin = pd.read_csv(ROOT_DIR/'allergy.csv.gz',compression = 'gzip')
# apachepatientresult_origin = pd.read_csv(ROOT_DIR/'apachePatientResult.csv.gz',compression = 'gzip')
# apachepredvar_origin = pd.read_csv(ROOT_DIR/'apachePredVar.csv.gz',compression = 'gzip')
# careplancareprovider_origin = pd.read_csv(ROOT_DIR/'carePlanCareProvider.csv.gz',compression = 'gzip')
# carePlanEOL_origin = pd.read_csv(ROOT_DIR/'carePlanEOL.csv.gz',compression = 'gzip')
# carePlanGeneral_origin = pd.read_csv(ROOT_DIR/'carePlanGeneral.csv.gz',compression = 'gzip')
# carePlanGoal_origin = pd.read_csv(ROOT_DIR/'carePlanGoal.csv.gz',compression = 'gzip')
# customLab_origin = pd.read_csv(ROOT_DIR/'customLab.csv.gz',compression = 'gzip')
# diagnosis_origin = pd.read_csv(ROOT_DIR/'diagnosis.csv.gz',compression = 'gzip')
# hospital_origin = pd.read_csv(ROOT_DIR/'hospital.csv.gz',compression = 'gzip')
# infusionDrug_origin = pd.read_csv(ROOT_DIR/'infusionDrug.csv.gz',compression = 'gzip')
intakeOutput_origin = pd.read_csv(ROOT_DIR/'intakeOutput.csv.gz',compression = 'gzip')
lab_origin = pd.read_csv(ROOT_DIR/'lab.csv.gz',compression = 'gzip')
# medication_origin = pd.read_csv(ROOT_DIR/'medication.csv.gz',compression = 'gzip')
# microLab_origin = pd.read_csv(ROOT_DIR/'microLab.csv.gz',compression = 'gzip')
# note_origin = pd.read_csv(ROOT_DIR/'note.csv.gz',compression = 'gzip')
# nurseAssessment_origin = pd.read_csv(ROOT_DIR/'nurseAssessment.csv.gz',compression = 'gzip')
# nurseCare_origin = pd.read_csv(ROOT_DIR/'nurseCare.csv.gz',compression = 'gzip')
# nurseCharting_origin = pd.read_csv(ROOT_DIR/'nurseCharting.csv.gz',compression = 'gzip')
# pastHistory_origin = pd.read_csv(ROOT_DIR/'pastHistory.csv.gz',compression = 'gzip')
patient_origin = pd.read_csv(ROOT_DIR/'patient.csv.gz',compression = 'gzip')
# physicalExam_origin = pd.read_csv(ROOT_DIR/'physicalExam.csv.gz',compression = 'gzip')
# carePlanGoal_origin = pd.read_csv(ROOT_DIR/'carePlanGoal.csv.gz',compression = 'gzip')
# respiratoryCare_origin = pd.read_csv(ROOT_DIR/'respiratoryCare.csv.gz',compression = 'gzip')
respiratoryCharting_origin = pd.read_csv(ROOT_DIR/'respiratoryCharting.csv.gz',compression = 'gzip')
# treatment_origin = pd.read_csv(ROOT_DIR/'treatment.csv.gz',compression = 'gzip')
vitalAperiodic_origin = pd.read_csv(ROOT_DIR/'vitalAperiodic.csv.gz',compression = 'gzip')
vitalPeriodic_origin = pd.read_csv(ROOT_DIR/'vitalPeriodic.csv.gz',compression = 'gzip')

C:\Users\korea\AppData\Local\Temp\ipykernel_10784\778987128.py:36: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  respiratoryCharting_origin = pd.read_csv(ROOT_DIR/'respiratoryCharting.csv.gz',compression = 'gzip')


## Select only used variables in dataframe

In [10]:
# 매핑 규칙 
"""
1. table > column > itemid 순서대로 매칭 시작 
2. 만약 column이 itemid가 아니면 해당 column 자체의 값을 활용 
3. method가 none이 아니면 단위 변환이 필요
"""

# 사용할 ITEMID LIST 정리
ITEMID_LIST = []
for iid in MAPPING_DF.itemid:
    if iid:
        if not isinstance(iid, list):
            iid = [iid]

        ITEMID_LIST += iid

print(ITEMID_LIST)

# intakeOutput은 regex까지 포함해서 처리
respiratory_df = respiratoryCharting_origin[respiratoryCharting_origin['respchartvaluelabel'].isin(ITEMID_LIST)]
outputevents_df = intakeOutput_origin[(intakeOutput_origin['celllabel'].isin(ITEMID_LIST)) | (intakeOutput_origin['celllabel'].str.contains('catheter.+output|output.+catheter', case = False, na = False))]
labevents_df = lab_origin[lab_origin['labname'].isin(ITEMID_LIST)]

gc.collect()

# 각 itemid마다 어떤 var_name으로 가야하는지 매핑
respiratory_df = pd.merge(respiratory_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'respchartvaluelabel', right_on='itemid')
outputevents_df = pd.merge(outputevents_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'celllabel',right_on = 'itemid')
labevents_df = pd.merge(labevents_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'labname',right_on = 'itemid')

gc.collect()

['albumin', 'alkaline phos.', 'ALT (SGPT)', 'AST (SGOT)', 'Base Excess', 'bicarbonate', 'total bilirubin', 'direct bilirubin', '-bands', 'BUN', 'calcium', 'ionized calcium', 'CPK', 'CPK-MB', 'chloride', 'creatinine', 'CRP', 'fibrinogen', 'FiO2', 'FiO2', 'glucose', 'bedside glucose', 'Hgb', 'PT - INR', 'potassium', 'lactate', '-lymphs', 'MCH', 'MCHC', 'MCV', 'Methemoglobin', 'magnesium', 'sodium', '-polys', 'O2 Sat (%)', 'paCO2', 'pH', 'phosphate', 'platelets x 1000', 'paO2', 'PTT', 'troponin - T', 'Urine', 'URINE CATHETER', 'WBC x 1000']


17

## Rename patientunitstayid columns

In [21]:
# patient unistayid -> stay_id
rename_cols = {'patientunitstayid' : 'stay_id'}

# Drugs 
# vasopressor_origin.rename(columns = rename_cols, inplace = True)
# fluids_origin.rename(columns = rename_cols, inplace = True)
# antibiotics_inf_origin.rename(columns = rename_cols, inplace = True)
# antibiotics_io_origin.rename(columns = rename_cols, inplace = True)

# Output
outputevents_df.rename(columns = rename_cols, inplace = True)


# Others
labevents_df.rename(columns = rename_cols, inplace = True)
respiratory_df.rename(columns = rename_cols, inplace = True)
vitalAperiodic_origin.rename(columns = rename_cols, inplace = True)
vitalPeriodic_origin.rename(columns = rename_cols, inplace = True)

## Exclusion Criteria & Preprocess Patient(admissionweight, admissionheight) Outlier  

In [56]:
## patient.csv 전처리.
# hospitaladmitoffset의 -를 바꾸면 병원 입원 후 몇 분만에 ICU에 입원했는지에 대한 시간이 나옴. 
# gender, ethnicity는 그냥 그 row 그대로 (patientunitstayid를 기준으로) 사용하자. 

icustays_df = patient_origin[['hospitalid', 'uniquepid', 'patienthealthsystemstayid', 'patientunitstayid',# ids 
                'hospitaladmitoffset', 'hospitaldischargeoffset', 'unitdischargeoffset', # times
                'age', 'gender', 'unittype', 'hospitaladmitsource', 'ethnicity', 'admissionheight', 'admissionweight', # variables
                'hospitaldischargestatus', 'unitdischargestatus', # mortality information
                ]].sort_values(['hospitalid', 'uniquepid', 'patienthealthsystemstayid', 'patientunitstayid', 'hospitaladmitoffset'])

icustays_df['hospitaldischargeoffset'] = icustays_df['hospitaldischargeoffset'] - icustays_df['hospitaladmitoffset']
icustays_df['los'] = icustays_df['unitdischargeoffset'] / 60 / 24
icustays_df.loc[icustays_df['age'] == '> 89', 'age'] = 90 # then 90

# 1. unitdischargestatus는 사망인데, hospitaldischargestatus는 살아있는 건 제외.
icustays_df = icustays_df[~((icustays_df['unitdischargestatus'] == 'Expired') 
                            & (icustays_df['hospitaldischargestatus'] == 'Alive'))].reset_index(drop = True)

# 2. unitdischargeoffset이 hospitaldischargeoffset보다 늦은 건 제외. 
icustays_df = icustays_df[icustays_df['unitdischargeoffset'] <= icustays_df['hospitaldischargeoffset']].reset_index(drop = True)

# 3. unitdischargeoffset이 0이거나 혹은 hospitaldischargeoffset이 0인건 제외 -> null값임.
icustays_df = icustays_df[(icustays_df['unitdischargeoffset']!=0) & (icustays_df['hospitaldischargeoffset']!=0)].reset_index(drop=True)

# 4. admissionweight, admissionheight이 결측이거나 극단적인 이상치라면 그냥 해당 환자 제거
# icustays_df = filter_outliers(icustays_df, 0.01, 0.99, value_col = ['admissionweight', 'admissionheight'])
wg_cond = (icustays_df['admissionweight'] >= MAPPING_DF[MAPPING_DF['var_name']=='weight']['normal_min'].values[0]) & (icustays_df['admissionweight'] <= MAPPING_DF[MAPPING_DF['var_name']=='weight']['normal_max'].values[0])
hg_cond = (icustays_df['admissionheight'] >= MAPPING_DF[MAPPING_DF['var_name']=='height']['normal_min'].values[0]) & (icustays_df['admissionheight'] <= MAPPING_DF[MAPPING_DF['var_name']=='height']['normal_max'].values[0])
icustays_df = icustays_df[(wg_cond) & (hg_cond)].reset_index(drop=True)

# 5. age, gender, hospitaldischargestatus, unitdischargestatus, hospitaladmitsource, ethnicity가 결측이면 해당 row 제외. (=MIMIC에서는 해당 row가 애초에 결측이 없음.)
icustays_df = icustays_df.dropna(subset=['age','gender','hospitaldischargestatus', 'unitdischargestatus', 'hospitaladmitsource', 'ethnicity']).reset_index(drop= True) 
icustays_df['age'] = icustays_df['age'].astype(int) # age columns -> int dtype

# Column Rename 
icustays_df.rename(columns = {'uniquepid':'subject_id', 'patienthealthsystemstayid':'hadm_id', 'patientunitstayid' : 'stay_id'}, inplace = True)

In [65]:
# Exclusion Criteria
def exclusion_criteria(df, min_los, min_age, max_age):
    
    # 1) Invalid LoS -> Maybe Zero
    df = df[df['los']>= 0]
    
    # 2) Min Los
    df = df[df['los']>= min_los]
    
    # 3) Min age : 18
    df = df[(df['age'] >= min_age) & df['age'] <= max_age]
    
    return df.reset_index(drop = True)

ex_icustays_df = exclusion_criteria(icustays_df, 1.25, 18, 89).sort_values(['subject_id', 'hadm_id', 'stay_id', 'unitdischargeoffset'])
print(f'{icustays_df.shape[0] - ex_icustays_df.shape[0]}명의 환자 삭제 최종 stay_id 개수 {ex_icustays_df.stay_id.nunique()} 명')

53984명의 환자 삭제 최종 stay_id 개수 78623 명


# 1) Exclusion Criteria & Checkout Valid Lab events & Outlier & Add time_since_ICU columns

## 1-1) Apply Exclusion Criteria

In [70]:
# offset 관련 이름 재설정.

# customLab, lab, nurseCharting, respiratoryCharting, urineoutput, vitalAperiodic, vitalPeriodic 

def apply_exclusion_criteria(df, ex_icustays_df, time_col_name : str = 'charttime', table_type : str = 'nurseCharting'):
    """
    Exclusion Criteria에 맞춰 invalid한 데이터 제거.
    
    ---- 적용 테이블 ----

    'outputevents'
    'respiratory'
    'labevents'
    'vitalAperiodic_origin'
    'vitalPeriodic_origin'
    """
    
    ## icustays에서 사용할 column만 추리기
    standard_merge_cols = 'stay_id'
    standard_icu_cols = ['subject_id','hadm_id','stay_id', 'unitdischargeoffset']
    
    merged_df = pd.merge(df, ex_icustays_df[standard_icu_cols], how = 'inner', on = standard_merge_cols).reset_index(drop = True) # inner로 겹치는 것들만 추리기.
    
    # charttime을 사용하는 경우
    if time_col_name == "charttime":
        
        if table_type == 'lab': # 이전과 다르게 ICU 입원 후의 Lab값만 사용
            merged_df = merged_df[(merged_df[time_col_name] >= 0) & (merged_df[time_col_name] <= merged_df['unitdischargeoffset'])].reset_index(drop=True) 
        
        else:
            merged_df = merged_df[(merged_df[time_col_name] >= 0) & (merged_df[time_col_name] <= merged_df['unitdischargeoffset'])].reset_index(drop=True) # charttime이 intime과 unitdischargeoffset 사이에 있는 값만 사용. 이외 제외.
        
        
    # charttime이 아닌 startime, endtime을 사용하는 경우   
    else: 
        merged_df = merged_df[~((merged_df['starttime'] < 0) & (merged_df['endtime'] <= 0))].reset_index(drop=True) # strattime, endtime이 intime보다 이전에 있는 경우 제외.
        
    print(f'Result {df.shape[0]} ---> {merged_df.shape[0]}, {df.shape[0] - merged_df.shape[0]} Delete!')
        
    return merged_df.drop(columns = ['unitdischargeoffset'])

## offset column명 변경.

# ### charttime으로 변경할 것.
respiratory_df.rename(columns = {'respchartoffset' : 'charttime'}, inplace = True)
labevents_df.rename(columns = {'labresultoffset' : 'charttime'}, inplace = True)
outputevents_df.rename(columns = {'intakeoutputoffset' : 'charttime'}, inplace = True)
vitalAperiodic_origin.rename(columns = {'observationoffset' : 'charttime'}, inplace = True)
vitalPeriodic_origin.rename(columns = {'observationoffset' : 'charttime'}, inplace = True)

# ## vital column명 최종 처리 Name으로 재정의
# aperiodic_rename_cols = { row['Mapping_col'] : row['Name'] for _,row in var_df[var_df['Linksto'] == 'vitalAperiodic'].iterrows()}
# periodic_rename_cols = { row['Mapping_col'] : row['Name'] for _,row in var_df[var_df['Linksto'] == 'vitalPeriodic'].iterrows()}

# vitalAperiodic_df.rename(columns = aperiodic_rename_cols, inplace = True)
# vitalPeriodic_df.rename(columns = periodic_rename_cols, inplace = True)

# start, end time

urine_df = apply_exclusion_criteria(outputevents_df, ex_icustays_df, 'charttime', 'urine')
del outputevents_df

respiratoryCharting_df = apply_exclusion_criteria(respiratory_df, ex_icustays_df, 'charttime', 'respiratoryCharting')
del respiratory_df

lab_df = apply_exclusion_criteria(labevents_df, ex_icustays_df, 'charttime', 'lab')
del labevents_df

vitalAperiodic_df = apply_exclusion_criteria(vitalAperiodic_origin, ex_icustays_df, 'charttime', 'vitalAperiodic')
del vitalAperiodic_origin

vitalPeriodic_df = apply_exclusion_criteria(vitalPeriodic_origin, ex_icustays_df, 'charttime', 'vitalPeriodic')
del vitalPeriodic_origin


Result 4205977 ---> 2303061, 1902916 Delete!
Result 6150616 ---> 3847344, 2303272 Delete!
Result 29196684 ---> 9856151, 19340533 Delete!
Result 25075074 ---> 15490042, 9585032 Delete!
Result 146671642 ---> 90799815, 55871827 Delete!


## 1-2) Unit Conversion

In [90]:
# 단위 변경해야할 것은 cai, crp뿐
display(MAPPING_DF[MAPPING_DF['method'].notna()])

display(respiratoryCharting_df[['var_name', 'itemid', 'unit']].drop_duplicates(), MAPPING_DF[MAPPING_DF['table'] == 'respiratorycharting'][['var_name', 'unit']])

display(pd.merge(lab_df[['var_name', 'itemid', 'labmeasurenamesystem']].drop_duplicates().sort_values('var_name'), MAPPING_DF[MAPPING_DF['table'] == 'lab'][['var_name', 'unit']].sort_values('var_name'), on='var_name'))

,var_name,normal_min,normal_max,category,table,column,itemid,unit,method,regex
0,age,0.0,100.0,demographics,patient,age,None,years,transform_fun(eicu_age),None
14,ca,4.0,20.0,chemistry,lab,labname,calcium,mg/dL,"convert_unit(binary_op(`*`, 4), 'mg/dL', 'mmol...",None
20,crp,0.0,NaN,chemistry,lab,labname,CRP,mg/L,"convert_unit(binary_op(`*`, 10), 'mg/L', 'mg/dl')",None
23,fio2,21.0,100.0,blood gas,respiratorycharting,respchartvaluelabel,FiO2,%,transform_fun(percent_as_numeric),None
24,fio2,21.0,100.0,blood gas,lab,labname,FiO2,%,"convert_unit(set_val(NA), '%', 'mm\\(hg\\)')",None
39,mg,0.5,5.0,chemistry,lab,labname,magnesium,mg/dL,"convert_unit(binary_op(`/`, 1.215), 'mg/dL', '...",None


,var_name,itemid,unit
0,fio2,FiO2,%


,var_name,unit
23,fio2,%


,var_name,itemid,labmeasurenamesystem,unit
0,alb,albumin,g/dL,g/dL
1,alp,alkaline phos.,Units/L,"[IU/L, U/l]"
2,alt,ALT (SGPT),Units/L,"[IU/L, U/l]"
3,ast,AST (SGOT),Units/L,"[IU/L, U/l]"
4,be,Base Excess,mEq/L,"[mEq/L, mmol/l]"
5,bicar,bicarbonate,mmol/L,"[mEq/L, mmol/l]"
6,bili,total bilirubin,mg/dL,mg/dL
7,bili_dir,direct bilirubin,mg/dL,mg/dL
8,bnd,-bands,%,%
9,bun,BUN,mg/dL,mg/dL


In [99]:
def crp_to_mgl(x):
    return x * 10

# calcium_ionized mg/dL -> mmol/L단위로 변경 -> 4로 나누면 됨.
def calcium_to_mmol(x):
    return x / 4


lab_df.loc[lab_df['var_name']=='crp', 'labresult'] = lab_df.loc[lab_df['var_name']=='crp', 'labresult'].apply(crp_to_mgl)
lab_df.loc[lab_df['var_name']=='cai', 'labresult'] = lab_df.loc[lab_df['var_name']=='cai', 'labresult'].apply(calcium_to_mmol)

## 1-3) Preprocess Outlier & Split to each purpose dataframe

- Drugs, Ventilator, Weight Dataframe을 제외한 나머지 데이터프레임에 대해서 이상치 처리. (Vital, Lab, UrineOutput, Score)

- 먼저, 각 데이터프레임의 value 값들을 수치형 변수로 바꿔줘야함.

- 동일한 변수더라도 테이블내에 이름이 다르거나 하면 다른 변수로 처리해서 우선 진행.

- 전처리 이후 MIMIC처럼 Chart , Lab 이렇게 통일시키는게 좋을듯

- vital관련 변수들은 melt해서 바꿔서 처리하거나 직접 값을 찾아서 해야할듯

#### Melting vitalAperiodic, vitalPeriodic 

In [168]:
def melt_vitaldf(df, value_vars):
    """
    vital관련 dataframe을 nurseCharting과 같이 변경하는 작업. -> 메모리가 터지니까 그냥 이상치 처리할 때 따로 value 값들만 넣어주기?
    """
    df = df.copy()
    
    df[value_vars] = df[value_vars].astype('float32') # 메모리 절약
    
    gc.collect()
    
    # 리스트로 지정한 컬럼만 녹임
    melted = df.melt(id_vars=['subject_id', 'hadm_id', 'stay_id', 'charttime'],
                    value_vars=value_vars,
                    var_name='label',
                    value_name='value')
    
    # 결측 제거
    melted = melted.dropna(subset=['value'])

    return melted

vitalAp_df = melt_vitaldf(vitalAperiodic_df, MAPPING_DF[MAPPING_DF['table']=='vitalaperiodic']['column'].values)
vitalP_df = melt_vitaldf(vitalPeriodic_df, MAPPING_DF[MAPPING_DF['table']=='vitalperiodic']['column'].values)

In [174]:
del vitalPeriodic_df
del vitalAperiodic_df

In [186]:
# vital 변수에 MAPPING_DF의 컬럼별 내용 추가
vitalAp_df = pd.merge(vitalAp_df, MAPPING_DF.explode('column', ignore_index=True)[['var_name', 'itemid', 'column', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'label', right_on='column')
vitalP_df = pd.merge(vitalP_df, MAPPING_DF.explode('column', ignore_index=True)[['var_name', 'itemid', 'column', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'label',right_on = 'column')


In [129]:
# urine output normal_min, normal_max 모두 할당하기
# urine에서 정규식으로 검색한 항목에는 min, max value가 들어가 있지 않아서 이를 추가로 부여함.
urine_df['normal_min'] = MAPPING_DF.query('var_name=="urine"')['normal_min'].unique()[0]
urine_df['normal_max'] = MAPPING_DF.query('var_name=="urine"')['normal_max'].unique()[0]

In [215]:
# Outlier Removal by min, max values
# urine_df, respiratoryCharting_df, lab_df, vitalPeriodic_df, vitalAperiodic_df

def filter_outliers(df, value_col):
    """
    연속형 변수 처리하기 위한 함수 
    """
    def _filtering(row):
        if not np.isnan(row['normal_min']) and not np.isnan(row['normal_max']):
            return row['normal_min'] <= row[value_col] <= row['normal_max'] 

        elif not np.isnan(row['normal_min']) and np.isnan(row['normal_max']): # min값만 있을 때
            return row['normal_min'] <= row[value_col]

        elif np.isnan(row['normal_min']) and not np.isnan(row['normal_max']): # max값만 있을 때 
            return row[value_col] <= row['normal_max'] 

        else:
            return True

    df_filtered = df.copy()
    outlier_binary = df.apply(_filtering, axis = 1)
    df_filtered['out_bin'] = outlier_binary

    df_filtered = df_filtered[df_filtered['out_bin']].reset_index(drop = True)

    return df_filtered

# 변환하기에 앞서 respiratoryCharting_df의 value값들을 float형으로 통일시켜줘야함. e.g. ~~% -> 숫자로만 표기
def keep_signed_number(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()

    # 맨 앞의 부호(- 또는 +)는 허용, 나머지는 숫자/소수점만 남김
    # 예: "-12.3%" -> "-12.3", "  45% " -> "45"
    s = s.replace(",", "")  # 천 단위 콤마가 있으면 제거(선택)

    # 문자열 어디에 있든 첫 번째 "부호 포함 숫자"를 추출
    import re
    m = re.search(r'[-+]?\d*\.?\d+', s)
    return float(m.group()) if m else pd.NA

respiratoryCharting_df['respchartvalue'] = respiratoryCharting_df['respchartvalue'].apply(keep_signed_number)


respiratoryCharting_df = filter_outliers(respiratoryCharting_df, 'respchartvalue')
lab_df = filter_outliers(lab_df, 'labresult')
urine_df = filter_outliers(urine_df, 'cellvaluenumeric')
vitalP_df = filter_outliers(vitalP_df, 'value')
vitalAp_df = filter_outliers(vitalAp_df, 'value')

gc.collect()

4445

# 2) Disease Annotation

## 2-1) AKI Annotation

- AKI Labeling은 입원 이전 7일 이내의 baseline을 구해야하기 때문에 original labevent dataframe으로 수행. 

### 2-1-1) Preparing Variables

In [ ]:
# # labevent에서는 creatinine을 사용
# def assign_stayid_to_lab(lab_event, ex_icustays_df, valid_days = 1):
#     valid_lab = lab_event[lab_event['hadm_id'].notna()].query('hadm_id.isin(@ex_icustays_df.hadm_id.unique())')

#     new_lab_rows = []
    
#     valid_lab['charttime'] = pd.to_datetime(valid_lab['charttime'], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['intime'] = pd.to_datetime(ex_icustays_df['intime'], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['outtime'] = pd.to_datetime(ex_icustays_df['outtime'], format="%Y-%m-%d %H:%M:%S")

#     valid_lab['stay_id'] = np.nan

#     for _, row in tqdm(valid_lab.iterrows(), total = valid_lab.shape[0], desc = 'assining_stay_id'):
#         standard_icu = ex_icustays_df[ex_icustays_df['hadm_id'] == row.hadm_id].sort_values('intime')
            
#         for _, stay in standard_icu.iterrows():
            
#             if stay['intime'] - pd.Timedelta(days=valid_days) <= row['charttime'] <= stay['outtime']: # 몇 일 전 데이터까지 사용할 것인지?
#                 row['stay_id'] = stay['stay_id']
#                 new_lab_rows.append(row)
#                 break
#             else:
#                 continue

#     result = pd.DataFrame(new_lab_rows)
    
#     return result

# creat_df = assign_stayid_to_lab(lab_df[lab_df['itemid'] == 'creatinine'].reset_index(drop=True), ex_icustays_df, 7) # 중환자실 입원 전 7일 이내의 데이터만 사용.

creat_df = lab_df[lab_df['itemid'] == 'creatinine'].reset_index(drop=True)
creat_df['unit'] = creat_df['unit'].astype('string')
creat_df.to_parquet(ROOT_DIR/'preprocessed_yaib'/f'creat_for_labeling_df.parquet', index=False)

# admission weight
adm_weights = ex_icustays_df[['stay_id', 'admissionweight']].reset_index(drop = True)

# 사용 변수 설정
common_cols = ['stay_id', 'hadm_id', 'charttime', 'labresult']

creat_df = creat_df[common_cols].sort_values(['stay_id', 'charttime']).reset_index(drop=True)

creat_df.rename(columns={'labresult' : 'creatinine'}, inplace = True)
adm_weights.rename(columns={'admissionweight' : 'admission_weight'}, inplace = True)


### 2-1-2) Caculate creatinine baseline & urine rate

In [228]:
# Exclusion Cohort용 baseline 계산

# 환자별로 우선 exclusion criteria를 위한 baseline 계산
def compute_baselines_for_excl(stay_df, creat_df):
    """
    YAIB 논문에 따르면 exclusion criteria를 계산하기 위해 입원 전 가장 최근값 or ICU 입원 후 가장 빠른 값이 4mg/dL이 넘으면 제외한다고 함.
    """
    
    def _baseline_preicu_last_or_earliest_in_icu(creat_df, icu_intime):
        # last prior to ICU
        prior = creat_df[creat_df['charttime'] < icu_intime]
        if not prior.empty:
            # 가장 최근값
            return prior.sort_values('charttime', ascending=False).iloc[0]['creatinine']
        
        # 없으면 ICU 내의 earliest
        in_icu = creat_df[creat_df['charttime'] >= icu_intime]

        if not in_icu.empty:
            return in_icu.sort_values('charttime').iloc[0]['creatinine']
        
        return np.nan
    
    rows = []
    for stay in tqdm(stay_df.stay_id.unique()):
        csub = creat_df[creat_df['stay_id'] == stay]
        icu_time = 0
        
        bpre = _baseline_preicu_last_or_earliest_in_icu(csub, icu_time)
        rows.append({'stay_id':stay, 'baseline_preicu_or_earliest': bpre})

    return pd.DataFrame(rows)

# baseline 계산 
baselines_df = compute_baselines_for_excl(ex_icustays_df, creat_df)

# exclusion mask 생성
aki_exclude_mask = baselines_df['baseline_preicu_or_earliest'] > 4.0
aki_excluded_stays = baselines_df.loc[aki_exclude_mask, 'stay_id'].tolist()


100%|██████████| 78623/78623 [00:41<00:00, 1902.56it/s]


In [237]:
# labeling 계산을 위한 baseline 계산

def compute_baseline_7d(group):
    """
    하나의 row마다 이전 7일 이내의 과거 값들 중 최소값을 baseline으로 설정
    """
    g = group.sort_values('charttime').copy()
    td = pd.to_timedelta(g['charttime'], unit='min')

    # set index -> rolling('7D') will consider previous 7 days up to current index
    g = group.set_index(td).sort_index()

    # rolling window of 7 days, exclude current row by closed='left'
    baseline_series = g['creatinine'].rolling('7D', closed='left').min()

    # baseline for earliest points will be NaN if no prior values
    g = g.assign(baseline_7d = baseline_series)

    return g.reset_index(drop=True)


baseline_7d_df = creat_df.groupby('stay_id', group_keys = False).apply(compute_baseline_7d)
baseline_7d_df = pd.merge(baseline_7d_df, baselines_df, how = 'left', on = 'stay_id')
baseline_7d_df['baseline_7d'] = baseline_7d_df['baseline_7d'].fillna(baseline_7d_df['baseline_preicu_or_earliest']) # 만약 baseline이 결측값이면 icu 입원 전 마지막 or icu 입원 후 첫 번째 값으로 채움

In [240]:
pd.to_timedelta(baseline_7d_df['charttime'], unit='min')

0         0 days 08:36:00
1         0 days 18:53:00
2         1 days 09:46:00
3         0 days 05:12:00
4         0 days 12:42:00
               ...       
404807    6 days 16:44:00
404808    7 days 16:18:00
404809    8 days 15:38:00
404810    9 days 16:50:00
404811   10 days 19:58:00
Name: charttime, Length: 404812, dtype: timedelta64[ns]

In [248]:
# Baseline 기준으로 flag 계산
tqdm.pandas()

def compute_baseline_rules(group):
    """
    rule 1 : 해당 시점 기준 48시간 이내 0.3mg/dL 이상 증가 여부
    rule 2 : baseline 대비 1.5배 이상 증가 여부
    """
    # group: 하나의 stay_id 그룹
    group = group.copy()
    td = pd.to_timedelta(group['charttime'], unit='min')
    group = group.set_index(td).sort_index()
    
    # 이전 row(즉, 현재 시점 제외)의 Creatinine 값을 기반으로 지난 48시간의 최소값을 계산
    group['min_creat_48h'] = group['creatinine'].shift(1).rolling('48h').min()
    group['rule1'] = (group['creatinine'] - group['min_creat_48h'] >= 0.3)
    group['rule1'] = group['rule1'].fillna(False)
    
    # Rule 2: 현재 Creatinine 값이 baseline_cr의 1.5배 이상인가?
    group['rule2'] = (group['creatinine'] >= group['baseline_7d'] * 1.5)
    
    return group.reset_index(drop=True)

aki_baseline_flag = baseline_7d_df.groupby('stay_id', group_keys=False).progress_apply(compute_baseline_rules)
aki_baseline_flag['AKI_Annotation'] = np.where((aki_baseline_flag['rule1'] | aki_baseline_flag['rule2']),1,0)

100%|██████████| 76231/76231 [01:15<00:00, 1006.33it/s]


In [255]:
# urine rate 계산
def cal_urine_rate(outputevent_df, adm_weight_df):

    df = outputevent_df.groupby(['stay_id', 'charttime'], as_index=False)['cellvaluenumeric'].sum() # 먼저 summation을 취함

    df = df.sort_values(['stay_id','charttime'])
    df['prev_time'] = df.groupby('stay_id')['charttime'].shift(1)

    df['elapsed_hours'] = (df['charttime'] - df['prev_time']) / 60 # 어차피 분 단위이니까 시간 단위로 바꾸려면 이렇게 60을 나누면 됨. 

    # apply rules: earliest -> 1h, cap at 24h, avoid zero
    df['elapsed_hours'] = df['elapsed_hours'].fillna(1.0)            # earliest = 1h
    df['elapsed_hours'] = df['elapsed_hours'].replace(0, 1.0)        # zero gap -> treat as 1h (or small eps)
    df['elapsed_hours'] = df['elapsed_hours'].clip(upper=24.0)       # max gap = 24h

    # 4) ml/hour 및 ml/kg/hour 계산
    df['ml_per_hour'] = df['cellvaluenumeric'] / df['elapsed_hours']

    # merge weight (use 75 kg if missing)
    df = df.merge(adm_weight_df[['stay_id','admission_weight']], on='stay_id', how='left')
    df['admission_weight'] = df['admission_weight'].fillna(75.0)

    df['urine_rate'] = df['ml_per_hour'] / df['admission_weight']

    return df

urine_rate_df = cal_urine_rate(outputevent_df=urine_df, adm_weight_df=adm_weights)
gc.collect()

18842

In [258]:
# 이 urine rate을 기반으로 가중 평균을 취해서 KDIGO 기준 annotation flag를 생성
def add_timeweighted_uo(group, window_hours=6, min_coverage_ratio=0.8):
    td = pd.to_timedelta(group['charttime'], unit='min')
    g = group.set_index(td).sort_index()

    # 1) interval별 volume 계산 (ml)
    g['volume_ml'] = g['urine_rate'] * g['admission_weight'] * g['elapsed_hours']

    # 2) 지난 6시간 동안 volume 합, 시간 합
    win = f'{window_hours}h'
    vol_roll = g['volume_ml'].rolling(win, min_periods=1)
    dt_roll  = g['elapsed_hours'].rolling(win, min_periods=1)

    g['vol_6h_sum']   = vol_roll.sum()
    g['hours_6h_sum'] = dt_roll.sum()

    # 3) 시간가중 평균 urine rate (mL/kg/h)
    #    UO_rate_6h = 총 volume / (weight * 실제 커버된 시간)
    g['uo_6h_timeweighted'] = (
        g['vol_6h_sum'] / (g['admission_weight'] * g['hours_6h_sum'])
    )

    # 4) KDIGO urine 기준: 6h 평균 < 0.5 mL/kg/h
    g['AKI_Annotation'] = (g['uo_6h_timeweighted'] < 0.5).astype(int)

    return g.reset_index(drop=True)

aki_urine_flag = (
    urine_rate_df
    .groupby('stay_id', group_keys=False)
    .apply(add_timeweighted_uo)
)

### 2-1-3) AKI Annotation

In [263]:
# AKI Annotation
def add_aki_annotation(baseline_flag, urine_flag, stays):
    """
    사용해야 하는 변수 : weight, urine output, 입원 이전 creatinine(baseline 계산용), 입원 이후 creatinine
    """

    use_cols = ['stay_id', 'charttime', 'AKI_Annotation']
    res_aki = pd.concat([baseline_flag[use_cols], urine_flag[use_cols]], axis = 0)

    print(f'Before Drop Duplicates : {res_aki.shape[0]} --> ', end = ' ')
    res_aki = res_aki.drop_duplicates()
    
    print(f'After Drop Duplicates : {res_aki.shape[0]}', end = '\n')

    return res_aki[res_aki['charttime'] >= 0][use_cols].reset_index(drop=True)

# Disease Annotation
aki_annot_df = add_aki_annotation(baseline_flag=aki_baseline_flag, urine_flag=aki_urine_flag, stays=ex_icustays_df)

gc.collect()

Before Drop Duplicates : 2697373 -->  After Drop Duplicates : 2684899


0

In [265]:
# Free up for memory
del baseline_7d_df, baselines_df, urine_rate_df, adm_weights
gc.collect()

7

# 3) Extract Final dataframe

- 해야할 작업 : 같은 charttime의 var이면 평균내기, 각 환자마다 Intime 이후 24시간까지의 값만 활용, demographic 정보까지 통합하기 

In [268]:
# rename
respiratoryCharting_df.rename(columns={'respchartvalue' : 'value'}, inplace = True)
lab_df.rename(columns={'labresult' : 'value'}, inplace = True)
urine_df.rename(columns={'cellvaluenumeric' : 'value'}, inplace = True)

In [272]:
def filter_until_24hours(df, stays, type):

    use_cols = ['stay_id', 'charttime', 'var_name', 'value']

    res = df[use_cols].copy()
    res['max_time'] = 60 * 24 # 60분 * 24 = 1 Days
    res = res[res['charttime'] <= res['max_time']]

    if type == 'output':
        # urine output은 같은 charttime인 경우 sum을 취하고
        res = res.groupby(['stay_id', 'charttime', 'var_name'],as_index = False)['value'].sum()
        res = res.sort_values(['stay_id', 'charttime'])
        
    else:
        # 나머지 변수 값들은 평균값으로 처리함.
        res = res.groupby(['stay_id', 'charttime', 'var_name'],as_index = False)['value'].mean()
        res = res.sort_values(['stay_id', 'charttime'])
        res.rename(columns={'valuenum' : 'value'}, inplace = True)

    return res.reset_index(drop=True)

filtered_respiratorycharting_df = filter_until_24hours(respiratoryCharting_df, ex_icustays_df, 'resp')
filtered_labevents_df = filter_until_24hours(lab_df, ex_icustays_df, 'lab')
filtered_outputevents_df = filter_until_24hours(urine_df, ex_icustays_df, 'output')
filtered_vitalP = filter_until_24hours(vitalP_df, ex_icustays_df, 'vital')
filtered_vitalAp = filter_until_24hours(vitalAp_df, ex_icustays_df, 'vital')


In [279]:
# static information
static_df = ex_icustays_df[['stay_id', 'age', 'gender', 'admissionweight', 'admissionheight']].rename(columns={'admissionweight' : 'weight', 'admissionheight' : 'height'}).melt(
    id_vars = ['stay_id'],
    value_vars = ['age', 'gender', 'weight', 'height'],
    var_name = 'var_name',
    value_name = 'value'
)

display(static_df.head())
display(static_df.var_name.unique())

,stay_id,var_name,value
0,157016,age,23
1,211144,age,67
2,221005,age,68
3,151900,age,66
4,218742,age,69


array(['age', 'gender', 'weight', 'height'], dtype=object)

In [282]:
filtered_respiratorycharting_df

,stay_id,charttime,var_name,value
0,141203,8,fio2,100.0
1,141203,81,fio2,100.0
2,141203,196,fio2,100.0
3,141203,311,fio2,100.0
4,141203,528,fio2,100.0
...,...,...,...,...
384839,3353251,1243,fio2,80.0
384840,3353251,1253,fio2,80.0
384841,3353251,1303,fio2,80.0
384842,3353251,1363,fio2,80.0


In [283]:
# Final 합치기 

dynamics_df = pd.concat([filtered_respiratorycharting_df, filtered_labevents_df, filtered_outputevents_df, filtered_vitalP, filtered_vitalAp], axis = 0).sort_values(['stay_id','charttime']).reset_index(drop=True)

In [285]:
del filtered_respiratorycharting_df, filtered_labevents_df, filtered_outputevents_df, filtered_vitalP, filtered_vitalAp
gc.collect()

7

# 4) Mortality, LoS, Disease Prediction Labeling

In [288]:
# Reference : An Extensive Data Preprocessing Pipeline for MIMIC-IV

# Reference : An Extensive Data Preprocessing Pipeline for MIMIC-IV
def add_mortality_status(x):
    if x == "Expired":
        return True
    else:
        return False

def add_inunit_mortality_to_icustays(stays):
    df = stays.copy()
    df['mortality_inunit'] = df['unitdischargestatus'].apply(add_mortality_status).astype(int)
    
    return df

def add_los_to_icustays(stays): 
    df = stays.copy()
    df['los_reg'] = ((df['los'] - 1) * 24).astype(int) 
    df['los_reg'] = df['los_reg'].clip(upper=168)
    
    return df

# 규민이형 말씀 : AKI를 training 할 때는 입원 후 하루 기간동안 AKI가 왔어도 사용한 경우가 있으나, 평가할 때는 빼는 게 맞는 거 같다는 생각.
def prediction_labeling(df : pd.DataFrame, annot_df : pd.DataFrame, prediction_hour : int = 8,
                        label_col_name : str = 'AKI_next_6h', annot_col_name : str = 'AKI_Annotation'):
    
    def _labeling(x):

        labels = [
            1 if x[annot_col_name].iloc[idx+1:idx+1+prediction_window].sum() >= 1 else 0
            for idx in range(len(x))
        ]
        x[label_col_name] = labels

        return x

    df['intime_next_24h'] = df['intime'] + pd.Timedelta(days=1)

    annot_df[[]]
    labeled_df = pd.merge(df, annot_df, how = 'left', on = ['stay_id', 'time_since_ICU'])
    labeled_df[annot_col_name] = labeled_df[annot_col_name].fillna(0)

    # generates sofa_next12h values within each stay, ensuring that predictions don’t span across different stays
    # The sofa_next12h column contains 1 if a future high SOFA score is anticipated and 0 otherwise, providing an actionable label for predictive modeling
    labeled_df = labeled_df.groupby('stay_id', group_keys=False).progress_apply(_labeling)

    return labeled_df

In [289]:
stays = ex_icustays_df.copy()

outcome_df = add_inunit_mortality_to_icustays(stays)
outcome_df = add_los_to_icustays(outcome_df)

# # # Diesease Early Prediction task
# ## 이 AKI를 어떻게 라벨링 할지 코드 다시 짜야함.
# preidiction_hour = 6 # 6h
# outcome_df = prediction_labeling(outcome_df, aki_annot_df, preidiction_hour, 'AKI_next_6h', 'AKI_Annotation') 

In [292]:
# AKI 라벨링 시 Cohort를 다르게 할지 아니면 그대로 할지 이런 것들도 고민해봐야 함.
aki_annot_df

,stay_id,charttime,AKI_Annotation
0,141168,516,0
1,141168,1133,1
2,141168,2026,1
3,141194,312,0
4,141194,762,0
...,...,...,...
2684894,3353251,15763,0
2684895,3353251,15823,0
2684896,3353251,15943,0
2684897,3353251,16123,0


In [293]:
dynamics_df.groupby('stay_id')['charttime'].count().describe()

count    78529.000000
mean      1051.005781
std        407.173907
min          1.000000
25%        857.000000
50%        938.000000
75%       1199.000000
max       2850.000000
Name: charttime, dtype: float64

In [295]:
# 저장할 데이터프레임들
static_df
dynamics_df
outcome_df
aki_annot_df

,stay_id,charttime,AKI_Annotation
0,141168,516,0
1,141168,1133,1
2,141168,2026,1
3,141194,312,0
4,141194,762,0
...,...,...,...
2684894,3353251,15763,0
2684895,3353251,15823,0
2684896,3353251,15943,0
2684897,3353251,16123,0


# 5) Final DataFrame

In [299]:
# Export Data
dynamics_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip', index=False)
static_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'static_df.csv.gz', compression='gzip', index=False)
outcome_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'outcome_df.csv.gz', compression='gzip', index=False)
aki_annot_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'aki_annot_df.csv.gz', compression='gzip', index=False)
np.save(ROOT_DIR/'preprocessed_yaib'/'aki_exclude_stayid.npy', np.array(aki_excluded_stays))

In [ ]:
ROOT_DIR = '/Users/korea/datasets/physionet.org/files/eicu-crd/2.0'
ROOT_DIR = Path(ROOT_DIR)

dynamics_df = pd.read_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip')

display(dynamics_df.groupby('stay_id')[['value']].count().describe())


# 6) Analysis of events

In [2]:
ROOT_DIR = '/Users/korea/EHRTTA/data/eicu'
ROOT_DIR = Path(ROOT_DIR)

# dynamics_df = pd.read_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip')
static_df = pd.read_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip')

# print(dynamics_df.var_name.nunique())
print(static_df.var_name.nunique())


4


C:\Users\korea\AppData\Local\Temp\ipykernel_7612\2507830160.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  static_df = pd.read_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip')


In [3]:
# static data의 anchor age, gender 변경
mapping = {'anchor_age' : 'age', 'gender' : 'sex'}
static_df['var_name'] = static_df['var_name'].replace(mapping)

static_df.to_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip', index = False)

In [6]:
lab_list = MAPPING_DF[~MAPPING_DF['category'].isin(['respiratory', 'vitals', 'output', 'demographics'])].var_name.unique()
vital_list = MAPPING_DF[MAPPING_DF['category'].isin(['respiratory', 'vitals'])].var_name.unique()
output_list = MAPPING_DF[MAPPING_DF['category'].isin(['output'])].var_name.unique()

lab = dynamics_df[dynamics_df['var_name'].isin(lab_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'lab'})
vitals = dynamics_df[dynamics_df['var_name'].isin(vital_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'vitals'})
output = dynamics_df[dynamics_df['var_name'].isin(output_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'output'})

analysis_df = pd.merge(lab, vitals, how='left', on ='stay_id').merge(output, how='left', on='stay_id')

analysis_df['total'] = analysis_df['lab'] + analysis_df['vitals'] + analysis_df['output']

for col in analysis_df.columns:
    if col != 'stay_id':
        print(f'----------------Number of {col} events----------------')
        display(analysis_df[col].describe())

----------------Number of lab events----------------


count    77389.000000
mean        42.850173
std         30.296840
min          1.000000
25%         22.000000
50%         35.000000
75%         56.000000
max        451.000000
Name: lab, dtype: float64

----------------Number of vitals events----------------


count    76504.000000
mean      1018.321356
std        378.226238
min          1.000000
25%        830.000000
50%        898.000000
75%       1165.000000
max       2779.000000
Name: vitals, dtype: float64

----------------Number of output events----------------


count    53172.000000
mean         8.472391
std          7.330422
min          1.000000
25%          2.000000
50%          6.000000
75%         13.000000
max         50.000000
Name: output, dtype: float64

----------------Number of total events----------------


count    52438.000000
mean      1034.783726
std        375.858359
min          3.000000
25%        856.000000
50%        935.000000
75%       1070.000000
max       2850.000000
Name: total, dtype: float64